<a href="https://colab.research.google.com/github/lizaoh/smp_program_data/blob/main/smp2015_extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Top of Script

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pymupdf
!pip install pymupdf-layout
!pip install pymupdf4llm
!pip install wordfreq
# !pip install rapidfuzz
import glob
import os
import pathlib
import pymupdf
import pymupdf.layout
import pymupdf4llm
import re
import pandas as pd
import unicodedata
import wordfreq
# from rapidfuzz import process, fuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 157.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: PyMuPDF
    Found existing installation: PyMuPDF 1.26.7
    Uninstalling PyMuPDF-1.26.7:
      Successfully uninstalled PyMuPDF-1.26.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.1/183.1 kB 15.1 MB/s eta 0:00:00


In [3]:
pdfs_path = '/content/drive/MyDrive/math_psych_work/Conference Programs/'

# Functions
Created with help from GPT 5.2, but some are my own code just turned into a function.

In [4]:
# Authors are always [last name], [first name, sometimes middle initial] and
# separated by commas, with last author having ", and" before it
AUTHOR_LINE_RE = re.compile(
    r"""
    ^
    # First author
    (?:[a-z]{2,}\s+)*[A-Z][a-zA-Z'-]+,      # last name (with lowercase particles)
    \s+
    (?:[A-Z][a-zA-Z'-]+|[A-Z]\.)            # first name OR initial
    (?:\s+(?:[A-Z][a-zA-Z'-]+|[A-Z]\.))*    # middle names / initials

    # Additional authors
    (?:,\s+
        (?:[a-z]{2,}\s+)*[A-Z][a-zA-Z'-]+,
        \s+
        (?:[A-Z][a-zA-Z'-]+|[A-Z]\.)
        (?:\s+(?:[A-Z][a-zA-Z'-]+|[A-Z]\.))*
    )*

    # Optional ", and Last, First"
    (?:,\s+and\s+
        (?:[a-z]{2,}\s+)*[A-Z][a-zA-Z'-]+,
        \s+
        (?:[A-Z][a-zA-Z'-]+|[A-Z]\.)
        (?:\s+(?:[A-Z][a-zA-Z'-]+|[A-Z]\.))*
    )?

    \.?
    $
    """,
    re.VERBOSE
)

def is_author_only_line(line: str) -> bool:
    return bool(AUTHOR_LINE_RE.match(line.strip()))

In [5]:
def parse_authors(line: str) -> list[str]:
    line = line.strip().rstrip('.')
    line = re.sub(r',?\s*and\s*$', '', line)
    line = line.replace(', and ', ', ')

    parts = [p.strip() for p in line.split(',') if p.strip()]

    if len(parts) % 2 != 0:
        # optional: log or inspect these cases
        parts = parts[:-1]

    return [
        f"{parts[i+1]} {parts[i]}"
        for i in range(0, len(parts), 2)
    ]

In [6]:
def rehyphenate_words(text, words_to_hyphenate):
    for word, hyphenated_word in words_to_hyphenate:
        text = text.replace(word, hyphenated_word)

    return text

In [7]:
def clean_text(text, words_to_hyphenate=None):
    if not text:
        return text

    if words_to_hyphenate:
      text = rehyphenate_words(text, words_to_hyphenate)

    text = re.sub(r'`"`\s(.*?)\s`"`', r'"\1"', text)  # Replace `"` with "
                                                      # and getting rid of extra
                                                      # space between the quotes
                                                      # and the first and last words

    text = re.sub(r' \n\n\d{1,3} \n\n', ' ', text)  # Remove page breaks with page number

    text = re.sub(r'-\s+(?!\b(?:and|or)\b)', '', text)  # Get rid of - and space after
                                                        # unless word after is
                                                        # "and" or "or"

    text = re.sub(r'\.\s*##.*$', '.', text,\
                  flags=re.DOTALL)           # Gets rid of extraneous text after
                                             # last sentence
    text = text.strip()
    text = fix_ligatures(text)

    return text

In [8]:
LIGATURE_MAP = {
    "ﬁ": "fi", "ﬂ": "fl", "ﬃ": "ffi", "ﬄ": "ffl", "ﬀ": "ff", "ﬅ": "ft", "ﬆ": "st",
    "Æ": "ffi", "¨u": "ü", "¨a": "ä", "´e": "é", "`e": "è", "`a": "à", "¨o": "ö",
    "˚a": "å", "c¸": "ç", '“': '"', '”': '"', "’": "'", '˜n': 'ñ', 'ˇs': 'š',
    "âĂŸ": "'"
}

def fix_ligatures(text):
    # Replace known ligatures
    for bad, good in LIGATURE_MAP.items():
        text = text.replace(bad, good)

    # Replace any private-use ligature (common in PDFs)
    cleaned_chars = []
    for ch in text:
        name = unicodedata.name(ch, "")
        if "LIGATURE" in name.upper():
            # Try to break it apart: remove spaces and lowercase
            base = name.split("LIGATURE")[-1]
            base = base.replace(" ", "").lower()
            cleaned_chars.append(base)
        else:
            cleaned_chars.append(ch)

    return "".join(cleaned_chars)

In [9]:
# Checks if valid word using Zipf frequency
def is_probably_valid(word, threshold=2.5):
    return wordfreq.zipf_frequency(word, "en") > threshold  # smaller number cuts off
                                                            # more words, bigger is
                                                            # more lenient

# Program

138 entries total (4 plenary talks, 24 symposium talks, 88 talks, and 22 posters)

Markdown shows bold and italic text here, but the extracted text doesn't show the authors. Author names are "last name, first name." Also no affiliations listed.


## Grab text from the pdf

In [10]:
year = '2015'
program = pymupdf.open(pdfs_path + f'smp{year}_program.pdf')

In [11]:
program_text = pymupdf4llm.to_markdown(program)

=== Document parser messages ===
Full-page OCR on page.number=0/1.



In [22]:
program_text[7100:9000]

"r all accepted submissions to the Meeting. This section is automatically generated from the submission website and may not reflect recent updates, such as withdrawn contributions or changes in authorship. \n\nFirst appear the plenary presentations, then the symposium presentations, then the regular submitted presentations, and finally the posters. \n\n## **Plenary presentations** \n\n## **Stability and plasticity in perceptual learning and models** \n\n## \n\nPerceptual learning improves how we see visual stimuli and is the basis of visual expertise. This talk considers some predictions and tests of an integrated reweighting framework (Dosher et al., 2013) for how perceptual learning balances plasticity and stability of visual representations and accounts for broad phenomena in perceptual learning related to the role of feedback, the challenges of learning intermixed tasks, and the role of bias in situations that are non-stationary due to learning. \n\n## **Reckoning with uncertainty:

In [23]:
# For getting author names because they don't show up in markdown one
page_text = []
for page in program:
  page_text.append(page.get_text('text'))

abstract_pages_text = page_text[6:]

In [24]:
abstract_pages_text[0]

"Abstracts\nBelow appear the abstracts for all accepted submissions to the Meeting.\nThis section is\nautomatically generated from the submission website and may not reﬂect recent updates,\nsuch as withdrawn contributions or changes in authorship.\nFirst appear the plenary presentations, then the symposium presentations, then the reg-\nular submitted presentations, and ﬁnally the posters.\nPlenary presentations\nStability and plasticity in perceptual learning and models\nDosher, Barbara\nPerceptual learning improves how we see visual stimuli and is the basis of visual exper-\ntise. This talk considers some predictions and tests of an integrated reweighting framework\n(Dosher et al., 2013) for how perceptual learning balances plasticity and stability of visual\nrepresentations and accounts for broad phenomena in perceptual learning related to the role\nof feedback, the challenges of learning intermixed tasks, and the role of bias in situations\nthat are non-stationary due to learning.\n

## Split text into presentation entries

In [25]:
all_abstracts = program_text.split('**Plenary presentations**')[1] # this is where abstracts start
split_abstracts = re.split(r'\n\n## \*\*', all_abstracts)
split_abstracts = [part for entry in split_abstracts for part in entry.split('\n\n**')]
abstract_entries = ['**' + entry.strip() for entry in split_abstracts if len(entry) > 200][:-1]

In [26]:
abstract_entries[:2]

['**Stability and plasticity in perceptual learning and models** \n\n## \n\nPerceptual learning improves how we see visual stimuli and is the basis of visual expertise. This talk considers some predictions and tests of an integrated reweighting framework (Dosher et al., 2013) for how perceptual learning balances plasticity and stability of visual representations and accounts for broad phenomena in perceptual learning related to the role of feedback, the challenges of learning intermixed tasks, and the role of bias in situations that are non-stationary due to learning.',
 "**Reckoning with uncertainty: Learning to discern and adapt to the obscure** \n\n## \n\nHow people should and do reckon with uncertainty is one of the most vexing problems in theories of choice. A number of solutions have been proposed. One is to simply assume, like Savage (1954) and Ramsey (1926) did, that for a 'rational' individual all uncertainties can be reduced to risks by replacing objective probabilities with 

## Find words to re-hyphenate

In [27]:
pattern = re.compile(r'([A-Za-z]+)-\n\s*([A-Za-z]+)')
possible_hyphenated_words = []

counter = 0
for p, page in enumerate(program[6:]):  # these are the pages with abstracts only
  text = fix_ligatures(page.get_text('text'))
  matches = pattern.findall(text)

  for left, right in matches:
    word = f"{left}{right}"
    hyphenated_word = f"{left}-{right}"
    if not is_probably_valid(word, threshold=1.6):
      possible_hyphenated_words.append([word, hyphenated_word])

      print(f"{counter:>3}: Page {p+7:<3} {hyphenated_word:<30} {word}")
      counter += 1

  0: Page 12  hyper-graphs                   hypergraphs
  1: Page 15  search-such                    searchsuch
  2: Page 16  supercapac-ity                 supercapacity
  3: Page 16  distribution-free              distributionfree
  4: Page 22  hyper-graphs                   hypergraphs
  5: Page 25  top-down                       topdown
  6: Page 29  exemplar-based                 exemplarbased
  7: Page 31  first-countable                firstcountable
  8: Page 31  D-convergence                  Dconvergence
  9: Page 32  finer-grainedness              finergrainedness
 10: Page 36  disen-tangles                  disentangles
 11: Page 40  two-dimensional                twodimensional
 12: Page 40  of-measure                     ofmeasure
 13: Page 41  similarity-and                 similarityand
 14: Page 42  attribute-based                attributebased
 15: Page 48  feature-based                  featurebased
 16: Page 50  multiple-processes             multipleprocesses
 17:

In [28]:
# Pick indices of words to rehyphenate
indices = [1, 3, (5, 9), (11, 20), (22, 24), (26, 30), 32]
words_to_hyphenate = [
    possible_hyphenated_words[i]
    for item in indices
    for i in ([item] if isinstance(item, int) else range(*item))
]

In [29]:
for word, hyphenated_word in words_to_hyphenate:
  print(f'{word:<30} {hyphenated_word}')

searchsuch                     search-such
distributionfree               distribution-free
topdown                        top-down
exemplarbased                  exemplar-based
firstcountable                 first-countable
Dconvergence                   D-convergence
twodimensional                 two-dimensional
ofmeasure                      of-measure
similarityand                  similarity-and
attributebased                 attribute-based
featurebased                   feature-based
multipleprocesses              multiple-processes
modelbased                     model-based
Crossvalidation                Cross-validation
Crossfitting                   Cross-fitting
fixedthreshold                 fixed-threshold
multiplechoice                 multiple-choice
speedaccuracy                  speed-accuracy
Sixtyeight                     Sixty-eight
reanalyzed                     re-analyzed
highdimensional                high-dimensional
primarystage                   primary-stag

## Sort authors, affiliations, title, and abstract

In [30]:
parsed_entries = []

for entry in abstract_entries:
  # Skip if just symposium session description
  if "Symposium on" in entry:
    continue
  cleaned_entry = clean_text(entry, words_to_hyphenate)
  entry_parts = cleaned_entry.split('\n\n', 2)

  # If no new lines before second line (where authors should be),
  # sort first part as info and second part as abstract
  if len(entry_parts) < 3:
    info_text = entry_parts[0]
    abstract = entry_parts[1]
  # Otherwise sort title, authors, abstract (no affiliations in this program)
  else:
    info_text = '\n\n'.join(entry_parts[:2])
    abstract = entry_parts[2]

  # Extracts title
  title_parts = re.findall(r'\*\*(.*?)\*\*', info_text)
  title = ' '.join(a.strip() for a in title_parts) if title_parts else None

  # Removes title from info_text to get authors
  authors_text = info_text

  for t in title_parts:
    authors_text = authors_text.replace(f'**{t}**', '')

  # Cleans up punctuation & whitespace
  authors_text = authors_text.replace('\n\n##', '')
  if authors_text.strip():
    authors = parse_authors(authors_text.strip())
    cleaned_authors = ', '.join(authors)
  else:
    cleaned_authors = ''

  parsed_entries.append({
    'year': year,
    'author(s)': cleaned_authors,
    'affiliation(s)': '',
    'title': title,
    'type': '',
    'abstract': abstract.strip()
  })

In [31]:
parsed_entries[:2]

[{'year': '2015',
  'author(s)': '',
  'affiliation(s)': '',
  'title': 'Stability and plasticity in perceptual learning and models',
  'type': '',
  'abstract': 'Perceptual learning improves how we see visual stimuli and is the basis of visual expertise. This talk considers some predictions and tests of an integrated reweighting framework (Dosher et al., 2013) for how perceptual learning balances plasticity and stability of visual representations and accounts for broad phenomena in perceptual learning related to the role of feedback, the challenges of learning intermixed tasks, and the role of bias in situations that are non-stationary due to learning.'},
 {'year': '2015',
  'author(s)': '',
  'affiliation(s)': '',
  'title': 'Reckoning with uncertainty: Learning to discern and adapt to the obscure',
  'type': '',
  'abstract': "How people should and do reckon with uncertainty is one of the most vexing problems in theories of choice. A number of solutions have been proposed. One is to

## Adding authors from normal text extraction (not markdown)

In [32]:
abstract_pages_text = [re.sub(r'\sand\s', ' and ', page) for page in abstract_pages_text]
abstract_pages_text = [page.replace(',\n', ', ') for page in abstract_pages_text]
abstract_pages_text = [page.replace('-\n', '') for page in abstract_pages_text]

In [33]:
authors_list = []
for page in abstract_pages_text:
  page_authors = []
  for line in page.split('\n'):
    if is_author_only_line(line):
      page_authors.append(line)
  if page_authors:
    for author in page_authors:
      page_authors = parse_authors(author)
      page_authors = ', '.join(page_authors)
      authors_list.append(page_authors)

# Delete the name listed for symposium descriptions
indices_to_delete = [4, 12, 18]
for k in sorted(indices_to_delete, reverse=True):
    del authors_list[k]

authors_list.insert(21, 'Joseph W. Houpt, Trisha Van Zandt, James T. Townsend')
authors_list.insert(48, 'Brandon Turner, Leendert Van Maanen, Birte Forstmann')
authors_list.insert(99, 'Luigi Lombardi, Antonio Calcagnì')
authors_list.insert(109, 'Charles A Doan, Ronaldo Vigo')

In [34]:
authors_list[:10]

['Barbara Dosher',
 'Ralph Hertwig',
 'Amy H Criss',
 'Joachim Vandekerckhove',
 'Christopher Olivola',
 'Rick Dale, David Vinson',
 'Michael Lee, Ravi Selker, Ravi Iyer',
 'Zita Oravecz',
 'Steven Piantadosi',
 'Jun Zhang']

In [130]:
# # For comparing authors (some were captured in markdown) and matching to titles
# for e, entry in enumerate(parsed_entries):
#   print(e)
#   print(entry['title'])
#   print(entry['author(s)'])
#   print(authors_list[e])
#   print('\n')

In [35]:
# Adds author from `author_list` if no author in entry
for e, entry in enumerate(parsed_entries):
  if not entry['author(s)']:
    entry['author(s)'] = authors_list[e]

In [36]:
parsed_entries[:2]

[{'year': '2015',
  'author(s)': 'Barbara Dosher',
  'affiliation(s)': '',
  'title': 'Stability and plasticity in perceptual learning and models',
  'type': '',
  'abstract': 'Perceptual learning improves how we see visual stimuli and is the basis of visual expertise. This talk considers some predictions and tests of an integrated reweighting framework (Dosher et al., 2013) for how perceptual learning balances plasticity and stability of visual representations and accounts for broad phenomena in perceptual learning related to the role of feedback, the challenges of learning intermixed tasks, and the role of bias in situations that are non-stationary due to learning.'},
 {'year': '2015',
  'author(s)': 'Ralph Hertwig',
  'affiliation(s)': '',
  'title': 'Reckoning with uncertainty: Learning to discern and adapt to the obscure',
  'type': '',
  'abstract': "How people should and do reckon with uncertainty is one of the most vexing problems in theories of choice. A number of solutions ha

# Create df and convert to csv

In [37]:
df = pd.DataFrame(parsed_entries, columns=["year", "author(s)", "affiliation(s)", "title", "type", "abstract"])

In [38]:
df.head()

,year,author(s),affiliation(s),title,type,abstract
0,2015,Barbara Dosher,,Stability and plasticity in perceptual learnin...,,Perceptual learning improves how we see visual...
1,2015,Ralph Hertwig,,Reckoning with uncertainty: Learning to discer...,,How people should and do reckon with uncertain...
2,2015,Amy H Criss,,Differentiation: A core mechanism underlying e...,,Differentiation originated in the perceptual l...
3,2015,Joachim Vandekerckhove,,Cognitive latent variable models,,"We introduce cognitive latent variable models,..."
4,2015,Christopher Olivola,,Harnessing Big Data to understand life-or-deat...,,How do people perceive and react to deadly eve...


In [39]:
df.to_csv(f"/content/drive/MyDrive/math_psych_work/csv/smp{year}_program.csv", index=False)